# Решение кейса от компании **"СИЛА"** **Автоматическая диспетчеризация заявок**

В этом ноутбуке мы рассмотрим задачу классификации отказов оборудования на основе текстовых описаний. Мы будем использовать предобученную модель BERT для обработки текстовых данных и предсказывать точки отказа и типы оборудования. Также реализуем извлечение серийного номера из текста.

## **Импорт необходимых библиотек**

In [ ]:
import pandas as pd
import numpy as np
import re
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.cuda.amp import GradScaler, autocast
import os


## **Установка устройства для вычислений**

Проверяем доступность GPU и устанавливаем соответствующее устройство для выполнения вычислений.

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Используемое устройство: {device}")


Используемое устройство: cuda


## **Задаём словарь для популярных слов**

Создаём словарь, чтобы модель точно могла понимать классификацию типа продукции

In [ ]:
typo_dict = {
    'ноутбку':'ноутбук', 'нобутук':'ноутбук', 'нотубук':'ноутбук', 'ноубтук':'ноутбук', 'ноутбу':'ноутбук', 'ноотбук':'ноутбук', 'наутбук':'ноутбук', 'нуотбук':'ноутбук', 'ноутьбук':'ноутбук', 'ноутбюк':'ноутбук', 'ноутбк':'ноутбук', 'нотбук':'ноутбук', 'нтубук':'ноутбук', 'нотуьбук':'ноутбук', 'ноубук':'ноутбук', 'нойтбук':'ноутбук', 'ноутбукк':'ноутбук', 'ноутбуук':'ноутбук', 'ноутубк':'ноутбук', 'ноутбукь':'ноутбук', 'нвотбук':'ноутбук', 'нюотбук':'ноутбук', 'оутбук':'ноутбук', 'ноытбук':'ноутбук', 'ноутюук':'ноутбук', 'нотубка':'ноутбук', 'нотубку':'ноутбук', 'нбоубук':'ноутбук', 'ноутубук':'ноутбук', 'нтуьбук':'ноутбук', 'нтбук':'ноутбук', 'нтобук':'ноутбук', 'ноубкук':'ноутбук', 'ноутубкк':'ноутбук', 'нботбук':'ноутбук', 'нотуб':'ноутбук', 'ноубтбук':'ноутбук', 'нтбк':'ноутбук', 'нобук':'ноутбук', 'нтубку':'ноутбук', 'нуотбк':'ноутбук', 'ноуьбук':'ноутбук', 'нотубка':'ноутбук', 'нотубк':'ноутбук', 'ноутбюук':'ноутбук', 'ноутуб':'ноутбук', 'ноутбкк':'ноутбук', 'нотбк':'ноутбук', 'нотубкка':'ноутбук', 'нобтбук':'ноутбук', 'нотюбук':'ноутбук', 'нтюбук':'ноутбук', 'нюьбук':'ноутбук', 'нотбкук':'ноутбук',

    'свервер':'сервер', 'сревер':'сервер', 'серер':'сервер', 'сервеер':'сервер', 'серве':'сервер', 'сеервер':'сервер', 'сервр':'сервер', 'сервар':'сервер', 'сервир':'сервер', 'серевр':'сервер', 'серевер':'сервер', 'сервре':'сервер', 'серверь':'сервер', 'серверр':'сервер', 'сервере':'сервер', 'сервис':'сервер', 'севрер':'сервер', 'севвер':'сервер', 'сервыер':'сервер', 'сервери':'сервер', 'сревре':'сервер', 'срвер':'сервер', 'сервен':'сервер', 'срввер':'сервер', 'сервёер':'сервер', 'серввёр':'сервер', 'север':'сервер', 'севр':'сервер', 'северь':'сервер', 'сервйер':'сервер', 'сервср':'сервер', 'севвёр':'сервер', 'сервёр':'сервер', 'сррвер':'сервер', 'северр':'сервер', 'серрвер':'сервер', 'серррвер':'сервер', 'серввер':'сервер', 'сервирр':'сервер', 'сервквер':'сервер'
}


## **Определение вспомогательных функций**

Здесь мы определяем несколько функций, которые помогут в обработке данных и оценке модели.

* `extract_serial_number`: извлекает серийный номер из текста
* `compare_serial_numbers`: сравнивает извлеченный и ожидаемый серийные номера.
* `evaluate_serial_numbers`: оценивает точность извлечения серийных номеров.
* `preprocess_text`: выполняет предварительную обработку текста.

In [ ]:
def extract_serial_number(text):
    """
    Извлекает серийный номер из текста. Если серийный номер не найден, возвращает 'Уточнить'.

    Параметры:
    text (str): Входной текст, из которого нужно извлечь серийный номер.

    Возвращает:
    str: Извлеченный серийный номер или 'Уточнить'.
    """
    if not isinstance(text, str):
        return 'Нет серийного номера'

    text = text.upper()

    cyrillic_to_latin = {
        ord('А'): 'A', ord('В'): 'B', ord('Е'): 'E', ord('К'): 'K',
        ord('М'): 'M', ord('Н'): 'H', ord('О'): 'O', ord('Р'): 'R',
        ord('С'): 'C', ord('Т'): 'T', ord('У'): 'Y', ord('Х'): 'X',
        ord('Ц'): 'C', ord('Ч'): 'CH', ord('Ш'): 'SH', ord('Щ'): 'SCH',
        ord('Ь'): '', ord('Ы'): 'Y', ord('Ъ'): '', ord('Э'): 'E',
        ord('Ю'): 'YU', ord('Я'): 'YA',

        ord('а'): 'A', ord('в'): 'B', ord('е'): 'E', ord('к'): 'K',
        ord('м'): 'M', ord('н'): 'H', ord('о'): 'O', ord('р'): 'R',
        ord('с'): 'C', ord('т'): 'T', ord('у'): 'Y', ord('х'): 'X',
        ord('ц'): 'C', ord('ч'): 'CH', ord('ш'): 'SH', ord('щ'): 'SCH',
        ord('ь'): '', ord('ы'): 'Y', ord('ъ'): '', ord('э'): 'E',
        ord('ю'): 'YU', ord('я'): 'YA',
    }

    text = text.translate(cyrillic_to_latin)

    pattern = r'\b[A-Z]{1,4}\d{6,}\b'
    matches = re.findall(pattern, text)

    if matches:
        return matches[0]
    else:
        return 'Уточнить'

def compare_serial_numbers(extracted, expected):
    """
    Сравнивает извлеченный и ожидаемый серийные номера.

    Параметры:
    extracted (str): Извлеченный серийный номер.
    expected (str): Ожидаемый серийный номер.

    Возвращает:
    bool: True, если серийные номера совпадают, иначе False.
    """
    if expected == 'Нет серийного номера':
        return extracted == 'Нет серийного номера'
    else:
        expected_serial = expected.upper().strip()
        extracted_serial = extracted.upper().strip()
        return expected_serial == extracted_serial

def evaluate_serial_numbers(true_serials, extracted_serials):
    """
    Оценивает точность извлечения серийных номеров.

    Параметры:
    true_serials (list): Список истинных серийных номеров.
    extracted_serials (list): Список извлеченных серийных номеров.

    Возвращает:
    float: Доля правильно извлеченных серийных номеров.
    """
    correct = 0
    total = len(true_serials)
    for true_serial, extracted in zip(true_serials, extracted_serials):
        if compare_serial_numbers(extracted, true_serial):
            correct += 1
    return correct / total if total > 0 else 0

def preprocess_text(text):
    """
    Выполняет предварительную обработку текста:
    - Удаляет специальные символы.
    - Приводит текст к нижнему регистру.
    - Удаляет лишние пробелы.

    Параметры:
    text (str): Исходный текст.

    Возвращает:
    str: Обработанный текст.
    """
    text = re.sub(r'_x000D_', ' ', str(text))
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    text = text.strip()

    # Заменяем опечатки
    words = text.split()
    corrected_words = []
    for word in words:
        corrected_word = typo_dict.get(word, word)
        corrected_words.append(corrected_word)
    corrected_text = ' '.join(corrected_words)

    return corrected_text

## **Загрузка и подготовка данных**

Загружаем датасет из CSV-файла и выполняем предварительную обработку данных для дальнейшего использования.


In [ ]:
dataset_path = 'dataset/dataset.csv'
data = pd.read_csv(
    dataset_path,
    sep=',',
    encoding='utf-8',
    on_bad_lines='skip',
    engine='python'
)

print(f"Количество загруженных строк: {len(data)}")

Количество загруженных строк: 10661


## **Объединение столбцов 'Тема' и 'Описание'**

Объединяем столбцы 'Тема' и 'Описание' в один текстовый столбец для удобства обработки.

In [ ]:
# Объединяем столбцы 'Тема' и 'Описание' в один текстовый столбец
data['text'] = data['Тема'].astype(str) + ' ' + data['Описание'].astype(str)
data['text'] = data['text'].apply(preprocess_text)

## **Обработка отсутствующих значений в 'Точке отказа'**

Заполняем пропущенные значения в столбце 'Точка отказа' значением 'Нет точки отказа', если такое значение отсутствует.

In [ ]:
# Обрабатываем отсутствующие значения в 'Точке отказа'
if 'Нет точки отказа' not in data['Точка отказа'].unique():
    data['Точка отказа'] = data['Точка отказа'].fillna('Нет точки отказа')

## **Кодирование целевых переменных**

Используем `LabelEncoder` для преобразования категориальных признаков в числовые значения, необходимые для обучения модели.

In [ ]:
# Кодируем целевые переменные
label_encoder_point = LabelEncoder()
label_encoder_type = LabelEncoder()

data['Точка отказа'] = label_encoder_point.fit_transform(data['Точка отказа'])
data['Тип оборудования'] = label_encoder_type.fit_transform(data['Тип оборудования'])

point_classes = label_encoder_point.classes_
type_classes = label_encoder_type.classes_

## **Инициализация токенизатора**

Используем предобученный токенизатор `DeepPavlov/rubert-base-cased` для преобразования текстовых данных в числовые тензоры.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## **Создание класса `CustomDataset`**

Определяем класс `CustomDataset`, который будет использоваться для загрузки данных в модель в формате, совместимом с PyTorch.

In [ ]:
class CustomDataset(Dataset):
    """
    Кастомный датасет для загрузки текстовых данных и соответствующих меток.
    """
    def __init__(self, texts, targets_point, targets_type, tokenizer, max_len):
        self.texts = texts
        self.targets_point = targets_point
        self.targets_type = targets_type
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        """
        Возвращает общее количество примеров в датасете.
        """
        return len(self.texts)

    def __getitem__(self, item):
        """
        Возвращает один пример из датасета.

        Параметры:
        item (int): Индекс примера.

        Возвращает:
        dict: Словарь с входными данными и метками.
        """
        text = str(self.texts[item])
        target_point = self.targets_point[item]
        target_type = self.targets_type[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'target_point': torch.tensor(target_point, dtype=torch.long),
            'target_type': torch.tensor(target_type, dtype=torch.long)
        }

## **Определение модели классификации**

Создаем класс `ClassificationModel`, который включает в себя предобученную модель BERT и дополнительные слои для классификации.

In [ ]:
class ClassificationModel(nn.Module):
    """
    Модель для классификации точки отказа и типа оборудования на основе выходов BERT.
    """
    def __init__(self, n_classes_point, n_classes_type):
        super(ClassificationModel, self).__init__()
        self.bert = AutoModel.from_pretrained('DeepPavlov/rubert-base-cased')
        self.drop = nn.Dropout(p=0.3)
        self.out_point = nn.Linear(self.bert.config.hidden_size, n_classes_point)
        self.out_type = nn.Linear(self.bert.config.hidden_size, n_classes_type)

        # Размораживаем все параметры BERT для обучения
        for param in self.bert.parameters():
            param.requires_grad = True

    def forward(self, input_ids, attention_mask):
        """
        Прямой проход модели.

        Параметры:
        input_ids (torch.Tensor): Идентификаторы входных токенов.
        attention_mask (torch.Tensor): Маска внимания.

        Возвращает:
        tuple: Логиты для точки отказа и типа оборудования.
        """
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        pooled_output = outputs.pooler_output
        output = self.drop(pooled_output)
        point_logits = self.out_point(output)
        type_logits = self.out_type(output)
        return point_logits, type_logits

## **Разделение данных на обучающую и валидационную выборки**

Используем функцию `train_test_split` для разделения данных на обучающую и валидационную выборки в соотношении 80/20.

In [ ]:
X_train, X_val, y_train_point, y_val_point, y_train_type, y_val_type = train_test_split(
    data['text'],
    data['Точка отказа'],
    data['Тип оборудования'],
    test_size=0.2,
    random_state=42
)

## **Подготовка `DataLoader` для обучения и валидации**

Создаем загрузчики данных (`DataLoader`) для обучения и валидации, что позволит эффективно передавать данные в модель.

In [ ]:
MAX_LEN = 512
BATCH_SIZE = 32
NUM_WORKERS = 4

# Создаем датасет и загрузчик для обучения
train_dataset = CustomDataset(
    X_train.values,
    y_train_point.values,
    y_train_type.values,
    tokenizer,
    MAX_LEN
)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True if torch.cuda.is_available() else False)

# Создаем датасет и загрузчик для валидации
val_dataset = CustomDataset(
    X_val.values,
    y_val_point.values,
    y_val_type.values,
    tokenizer,
    MAX_LEN
)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True if torch.cuda.is_available() else False)

## **Определение функций обучения и оценки модели**

Создаем функции `train_epoch` и `eval_model`, которые будут использоваться для обучения модели и оценки ее качества на валидационном наборе.

In [ ]:
def train_epoch(model, data_loader, loss_fn, optimizer, device, scaler):
    """
    Обучает модель на одной эпохе.

    Параметры:
    model (nn.Module): Обучаемая модель.
    data_loader (DataLoader): Загрузчик данных для обучения.
    loss_fn (nn.Module): Функция потерь.
    optimizer (torch.optim.Optimizer): Оптимизатор.
    device (torch.device): Устройство для вычислений.
    scaler (GradScaler): Скалер для AMP.

    Возвращает:
    float: Среднее значение функции потерь за эпоху.
    """
    model.train()
    losses = []

    for d in data_loader:
        input_ids = d['input_ids'].to(device, non_blocking=True)
        attention_mask = d['attention_mask'].to(device, non_blocking=True)
        target_point = d['target_point'].to(device, non_blocking=True)
        target_type = d['target_type'].to(device, non_blocking=True)

        optimizer.zero_grad()

        with autocast():
            outputs_point, outputs_type = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            loss_point = loss_fn(outputs_point, target_point)
            loss_type = loss_fn(outputs_type, target_type)
            loss = loss_point + loss_type

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        losses.append(loss.item())

    return np.mean(losses)

def eval_model(model, data_loader, loss_fn, device):
    """
    Оценивает модель на валидационном наборе.

    Параметры:
    model (nn.Module): Обученная модель.
    data_loader (DataLoader): Загрузчик данных для валидации.
    loss_fn (nn.Module): Функция потерь.
    device (torch.device): Устройство для вычислений.

    Возвращает:
    tuple: Среднее значение функции потерь, предсказания и истинные метки.
    """
    model.eval()
    losses = []
    predictions_point = []
    predictions_type = []
    real_point = []
    real_type = []

    with torch.no_grad():
        for d in data_loader:
            input_ids = d['input_ids'].to(device, non_blocking=True)
            attention_mask = d['attention_mask'].to(device, non_blocking=True)
            target_point = d['target_point'].to(device, non_blocking=True)
            target_type = d['target_type'].to(device, non_blocking=True)

            with autocast():
                outputs_point, outputs_type = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask
                )
                loss_point = loss_fn(outputs_point, target_point)
                loss_type = loss_fn(outputs_type, target_type)
                loss = loss_point + loss_type
                losses.append(loss.item())

            _, preds_point_batch = torch.max(outputs_point, dim=1)
            _, preds_type_batch = torch.max(outputs_type, dim=1)

            predictions_point.extend(preds_point_batch.cpu().numpy())
            predictions_type.extend(preds_type_batch.cpu().numpy())
            real_point.extend(target_point.cpu().numpy())
            real_type.extend(target_type.cpu().numpy())

    return np.mean(losses), predictions_point, predictions_type, real_point, real_type

## **Функция предсказания для нового текста**

Определяем функцию `predict`, которая принимает на вход новый текст и возвращает предсказания модели вместе с извлеченным серийным номером.

In [ ]:
def predict(text, model, tokenizer, label_encoder_point, label_encoder_type, device, max_len=512):
    """
    Делает предсказание для нового текста.

    Параметры:
    text (str): Входной текст.
    model (nn.Module): Обученная модель.
    tokenizer (AutoTokenizer): Токенизатор.
    label_encoder_point (LabelEncoder): Кодировщик меток для точки отказа.
    label_encoder_type (LabelEncoder): Кодировщик меток для типа оборудования.
    device (torch.device): Устройство для вычислений.
    max_len (int): Максимальная длина последовательности.

    Возвращает:
    tuple: Предсказанные метки для точки отказа и типа оборудования, а также извлеченный серийный номер.
    """
    model.eval()
    processed_text = preprocess_text(text)

    encoding = tokenizer.encode_plus(
        processed_text,
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )

    input_ids = encoding['input_ids'].to(device, non_blocking=True)
    attention_mask = encoding['attention_mask'].to(device, non_blocking=True)

    with torch.no_grad():
        outputs_point, outputs_type = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

    _, preds_point = torch.max(outputs_point, dim=1)
    _, preds_type = torch.max(outputs_type, dim=1)

    point_label = label_encoder_point.inverse_transform(preds_point.cpu().numpy())[0]
    type_label = label_encoder_type.inverse_transform(preds_type.cpu().numpy())[0]

    serial_number = extract_serial_number(text)

    return point_label, type_label, serial_number

## **Инициализация и обучение модели**

Создаем экземпляр модели, определяем оптимизатор и функцию потерь. Затем запускаем процесс обучения модели на протяжении нескольких эпох.

In [ ]:
n_classes_point = len(point_classes)
n_classes_type = len(type_classes)

model = ClassificationModel(n_classes_point, n_classes_type)
model = model.to(device)

if torch.cuda.device_count() > 1:
    print(f"Используем {torch.cuda.device_count()} GPU")
    model = nn.DataParallel(model)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = nn.CrossEntropyLoss().to(device)
scaler = GradScaler()

EPOCHS = 10

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    train_loss = train_epoch(
        model,
        train_loader,
        loss_fn,
        optimizer,
        device,
        scaler
    )
    print(f'Train loss: {train_loss}')

    val_loss, preds_point, preds_type, real_point, real_type = eval_model(
        model,
        val_loader,
        loss_fn,
        device
    )
    print(f'Validation loss: {val_loss}')

    f1_point = f1_score(real_point, preds_point, average='weighted')
    f1_type = f1_score(real_type, preds_type, average='weighted')
    print(f'F1 Score Точка отказа: {f1_point}')
    print(f'F1 Score Тип оборудования: {f1_type}')

    val_texts = X_val.values
    val_true_serials = data.loc[X_val.index, 'Серийный номер'].values
    val_extracted_serials = [extract_serial_number(text) for text in val_texts]
    serial_accuracy = evaluate_serial_numbers(val_true_serials, val_extracted_serials)
    print(f'Accuracy для серийного номера: {serial_accuracy}')

    average_metric = (f1_point + f1_type + serial_accuracy) / 3
    print(f'Средняя метрика: {average_metric}\n')

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1/10


<ipython-input-51-e4f407656798>:13: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
<ipython-input-49-ffa3729ce326>:27: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Train loss: 1.6069507066127473


<ipython-input-49-ffa3729ce326>:71: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


...

Validation loss: 0.3212786698550509
F1 Score Точка отказа: 0.9213119599825605
F1 Score Тип оборудования: 0.9988920491277361
Accuracy для серийного номера: 1.0
Средняя метрика: 0.9734013363700988

Epoch 10/10


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-49-ffa3729ce326>:27: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Train loss: 0.07331951405951935


<ipython-input-49-ffa3729ce326>:71: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Validation loss: 0.35019769852882937
F1 Score Точка отказа: 0.9158359536282206
F1 Score Тип оборудования: 0.9988920491277361
Accuracy для серийного номера: 1.0
Средняя метрика: 0.9715760009186521



## **Сохранение обученной модели**

После обучения сохраняем модель для последующего использования, используя `torch.jit` для создания скриптованной версии модели.

In [ ]:
# Определяем путь для сохранения модели
model_save_dir = os.path.dirname(dataset_path)
model_save_path = os.path.join(model_save_dir, 'model.pt')

try:
    # Подготовка примерного входа для трассировки
    example_input_ids = torch.randint(0, tokenizer.vocab_size, (1, MAX_LEN)).to(device)
    example_attention_mask = torch.ones((1, MAX_LEN), dtype=torch.long).to(device)

    # Трассировка модели
    scripted_model = torch.jit.trace(model, (example_input_ids, example_attention_mask))

    # Сохранение трассированной модели
    torch.jit.save(scripted_model, model_save_path)

    print(f"Модель сохранена по пути: {model_save_path}")

except Exception as e:
    print(f"Не удалось сохранить модель с помощью torch.jit: {e}")
    # Альтернативно, сохраняем state_dict модели
    model_state_dict_path = os.path.join(model_save_dir, 'model_state_dict.pth')
    torch.save(model.state_dict(), model_state_dict_path)
    print(f"Сохранена state_dict модели по пути: {model_state_dict_path}")


In [ ]:
import joblib

joblib.dump(label_encoder_point, 'label_encoder_point.joblib')
joblib.dump(label_encoder_type, 'label_encoder_type.joblib')

## **Пример использования обученной модели**

Показываем, как можно использовать сохраненную модель для предсказания на новом тексте.

In [ ]:
# Пример использования модели
new_text = """
Привет. Пришел конец моему ноутбуку ABCD123456.
"""

point_label, type_label, serial_number = predict(
    new_text,
    model,
    tokenizer,
    label_encoder_point,
    label_encoder_type,
    device
)

print(f'Точка отказа: {point_label}')
print(f'Тип оборудования: {type_label}')
print(f'Серийный номер: {serial_number}')
